# <font color='blue'>Introduction to PySpark
---

- In this Notebook, we will learn how to use and operate Spark using Python called PySpark
- To make sure PySpark is already installed on your machine, we can import `pyspark` and check the version

In [1]:
# import library
import pyspark

In [2]:
# check pyspark version
pyspark.__version__

'3.5.0'

- Ok, we're already importing `pyspark` then how do we use the function and component inside pyspark?
- To use all the functions and components inside pyspark, we must create `SparkSession`
- To create `SparkSession` we can import this function `from pyspark.sql import SparkSession`
- Then, if we want to create `SparkSession` we can use this code snippet
  
```python
spark = SparkSession \
    .builder \
    .appName("...") \ # you can insert what kind of appName that you want
    .getOrCreate()
```

In [3]:
from pyspark.sql import SparkSession

In [4]:
spark = SparkSession.builder.appName("Learn PySpark Data Pipeline").getOrCreate()

In [5]:
spark

### SparkContext UI
---

- When we're creating SparkContext, we can get a web-based User Interface that will give us information about Spark execution, like run time, the process being split into many tasks, etc
- It will help us to debug, check our Spark performance, and many more
- To access SparkContext UI, we can access http://localhost:4040
- Make sure, to access SparkContext UI you're already create SparkSession
- If you're already creating SparkSession, you can click button `Spark UI`

<center>
    <img src="https://sekolahdata-assets.s3.ap-southeast-1.amazonaws.com/notebook-images/mde-data-ingestion-spark/spark_ui.png">
</center>

You can try to run this code below and see the Spark UI

In [7]:
df_rdd = spark.sparkContext\
            .parallelize([(1, 2, 3, 'a b c'),
                            (4, 5, 6, 'd e f'),
                            (7, 8, 9, 'g h i')])\
            .toDF(['col_1', 'col_2', 'col_3', 'col_4'])

df_rdd.show()

+-----+-----+-----+-----+
|col_1|col_2|col_3|col_4|
+-----+-----+-----+-----+
|    1|    2|    3|a b c|
|    4|    5|    6|d e f|
|    7|    8|    9|g h i|
+-----+-----+-----+-----+



In Spark UI will show what jobs are currently run by PySpark

<center>
    <img src="https://sekolahdata-assets.s3.ap-southeast-1.amazonaws.com/notebook-images/mde-data-ingestion-spark/spark_ui_run.png">
</center>

# <font color='blue'>Pandas vs PySpark
---

- In the previous section, we already learned that PySpark is similar to Pandas 
- Especially for data processing, we can:
    - Read Data
    - Select Data
    - Filter Data
    - Transform Data
    - etc
- We're going to learn about how to do it using PySpark in this course
- But what's the difference between them?
- The main difference between them is that PySpark will split the task!
- It will affect in speed time when reading data
- In this section, we're going to demonstrate it to see the speed difference
- You will see the difference if using data that have many records

---
- We will try to read data that have more than 5 million rows
- Say we have transaction data about Flights data, you can access the data in here [dataset](https://drive.google.com/file/d/1GggQUGzWFeLXwITGKN0s_NNmkep9V-WQ/view?usp=drive_link)
- We will compare it with Pandas and PySpark

**Pandas**
---

In [10]:
import pandas as pd
import time

In [23]:
DATA_PATH = "../data/"

In [24]:
# we're going to compare the speed 

def read_big_table_pandas(filename: str):
    start_time = time.time()
    
    df = pd.read_csv(DATA_PATH + filename)
    
    end_time = time.time()
    
    elapsed_time = end_time - start_time
    
    return elapsed_time, df

In [25]:
elapsed_time, df_pandas = read_big_table_pandas(filename = "bank_churners.csv")
print(f"Time to read CSV: {elapsed_time} seconds")

Time to read CSV: 0.04840540885925293 seconds


In [26]:
df_pandas.head()

,Attrition_Flag,Customer_Age,Gender,Dependent_count,Education_Level,Marital_Status,Income_Category,Card_Category,Months_on_book,Total_Relationship_Count,Months_Inactive_12_mon,Contacts_Count_12_mon,Credit_Limit,Total_Revolving_Bal,Avg_Open_To_Buy,Total_Amt_Chng_Q4_Q1,Total_Trans_Amt,Total_Trans_Ct,Total_Ct_Chng_Q4_Q1,Avg_Utilization_Ratio
0,Existing Customer,45,M,3,High School,Married,$60K - $80K,Blue,39,5,1,3,12691.0,777,11914.0,1.335,1144,42,1.625,0.061
1,Existing Customer,49,F,5,Graduate,Single,Less than $40K,Blue,44,6,1,2,8256.0,864,7392.0,1.541,1291,33,3.714,0.105
2,Existing Customer,51,M,3,Graduate,Married,$80K - $120K,Blue,36,4,1,0,3418.0,0,3418.0,2.594,1887,20,2.333,0.000
3,Existing Customer,40,F,4,High School,Unknown,Less than $40K,Blue,34,3,4,1,3313.0,2517,796.0,1.405,1171,20,2.333,0.760
4,Existing Customer,40,M,3,Uneducated,Married,$60K - $80K,Blue,21,5,1,0,4716.0,0,4716.0,2.175,816,28,2.500,0.000


**PySpark**
---

In [27]:
def read_big_table_pyspark(filename: str):
    start_time = time.time()
    
    df = spark \
        .read \
        .option("header", "true")\
        .csv(DATA_PATH + filename)
    
    end_time = time.time()
    
    elapsed_time = end_time - start_time
    
    return elapsed_time, df

In [28]:
elapsed_time, df_pyspark = read_big_table_pyspark(filename = "bank_churners.csv")
print(f"Time to read CSV: {elapsed_time} seconds")

Time to read CSV: 0.3867511749267578 seconds


In [29]:
df_pyspark.show(5)

+-----------------+------------+------+---------------+---------------+--------------+---------------+-------------+--------------+------------------------+----------------------+---------------------+------------+-------------------+---------------+--------------------+---------------+--------------+-------------------+---------------------+
|   Attrition_Flag|Customer_Age|Gender|Dependent_count|Education_Level|Marital_Status|Income_Category|Card_Category|Months_on_book|Total_Relationship_Count|Months_Inactive_12_mon|Contacts_Count_12_mon|Credit_Limit|Total_Revolving_Bal|Avg_Open_To_Buy|Total_Amt_Chng_Q4_Q1|Total_Trans_Amt|Total_Trans_Ct|Total_Ct_Chng_Q4_Q1|Avg_Utilization_Ratio|
+-----------------+------------+------+---------------+---------------+--------------+---------------+-------------+--------------+------------------------+----------------------+---------------------+------------+-------------------+---------------+--------------------+---------------+--------------+--------

# <font color='blue'>Read Data using PySpark
---

- Just like in Pandas, when using PySpark we can read data from various formats like:
    - File
    - Database
    - JSON
    - API
    - etc
- For more detailed what kind of format data that PySpark can read, you can check in this [documentation](https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/io.html)
- PySpark also can treat the data into DataFrame

---
In this section, we're going to learn how to read data from csv file and Postgres database 

**Read csv file**
---

- Say, we're using Flights Transactional Data
- The file in directory `data/bank_churners.csv`
- To read data using PySpark, first we can use `SparkSession` that we've already created
- So, for the next of this course we will use the variable `spark`
- The snippet code will be like this

```python
df = spark.read.option("header", "true").FORMAT_DATA(...) 
```

- In `FORMAT_DATA`we can adjust with the format data that we have
- So, in this case if we're going to read the `csv` file the `FORMAT_DATA` will be `csv`
- Then, inside `csv` we can put the filename of the data

```python
df = spark.read.csv(filename)
```

In [7]:
df = spark.read.option("header", "true").csv("../data/bank_churners.csv")

In [8]:
df.show()

+-----------------+------------+------+---------------+---------------+--------------+---------------+-------------+--------------+------------------------+----------------------+---------------------+------------+-------------------+---------------+--------------------+---------------+--------------+-------------------+---------------------+
|   Attrition_Flag|Customer_Age|Gender|Dependent_count|Education_Level|Marital_Status|Income_Category|Card_Category|Months_on_book|Total_Relationship_Count|Months_Inactive_12_mon|Contacts_Count_12_mon|Credit_Limit|Total_Revolving_Bal|Avg_Open_To_Buy|Total_Amt_Chng_Q4_Q1|Total_Trans_Amt|Total_Trans_Ct|Total_Ct_Chng_Q4_Q1|Avg_Utilization_Ratio|
+-----------------+------------+------+---------------+---------------+--------------+---------------+-------------+--------------+------------------------+----------------------+---------------------+------------+-------------------+---------------+--------------------+---------------+--------------+--------

**Read Database data**
---

- To read data from the database, first we must own the database driver
- Because PySpark is run in Scala and Java, so our database driver must be in `jar` files
- If you're using PySpark Docker from Pacmann's repo, we've already included `jar` files in that repo
- Next step, we can copy the `jar` files into our docker container by running this command in the terminal
  
  `docker cp driver/postgresql-42.6.0.jar pyspark_container:/usr/local/spark/jars/postgresql-42.6.0.jar`

- After we set up the database driver, we can start reading the database by using PySpark

---
- Say, we're already have a database from our docker compose and this is our config for database and we want to read `payments` table

```
DB_URL = jdbc:postgresql://pachotel_db_container:5432/pachotel
DB_TABLE = payment
DB_USER = postgres
DB_PASS = cobapassword
```

In [6]:
# set variable for database

DB_URL = "jdbc:postgresql://pachotel_db_container:5432/pachotel"
DB_TABLE = "payment" 
DB_USER = "postgres"
DB_PASS = "cobapassword"

In [7]:
# set config
jdbc_url = DB_URL
table_name = DB_TABLE
connection_properties = {
    "user": DB_USER,
    "password": DB_PASS,
    "driver": "org.postgresql.Driver" # set driver postgres
}

- To read the database using PySpark, we can use `jdbc`
- Then, we set parameters for:
    - `url`: URL for database
    - `table`: table name that we want to access
    - `properties`: database config

In [8]:
df = spark.read.jdbc(url=jdbc_url, table=table_name, properties=connection_properties)

# Display the DataFrame
df.show()

+----------+--------------+--------+--------------+--------------+--------------------+--------------------+--------------------+
|payment_id|reservation_id|provider|account_number|payment_status|        payment_date|         expire_date|          created_at|
+----------+--------------+--------+--------------+--------------+--------------------+--------------------+--------------------+
|         1|             1|     Ovo|     038137149|       Success|2020-10-20 17:07:...|2020-10-28 18:10:...|2024-02-09 13:59:...|
|         2|             2|     BCA|     042103729|       Success|2015-02-28 16:44:...|2015-03-08 05:27:...|2024-02-09 13:59:...|
|         3|             3| Permata|     058689635|       Success|2022-04-03 13:27:...|2022-04-06 14:20:...|2024-02-09 13:59:...|
|         4|             4|     BNI|     107161965|       Success|2019-10-04 08:39:...|2019-10-08 02:01:...|2024-02-09 13:59:...|
|         5|             5|     BSI|     014334131|        Failed|                NULL|202

- We've already select and filtered data based on the requirements given to us
- The next step is to **store the results** or export the data
- By using PySpark, we can save the output into a file or database
- When export the data we can use method `write` then followed by format data we want

**Save to File**
---

- In this case, we will try to save the output into CSV files
- When exporting the file into CSV by using PySpark, we can choose the output format:
    - can be only one file: `output.csv`
    - split the output: `output_1.csv`, `output_2.csv`, `output_n.csv`

---
- Given the flight dataset, we're going to selecting and filtering the data
- Selected columns:
    - `ACCOUNT_NUMBER`
    - `PAYMENT_STATUS`
    - `PAYMENT_DATE`
    - `EXPIRE_DATE`
    - `PROVIDER`

In [30]:
SELECTED_COLS = ["ACCOUNT_NUMBER", "PAYMENT_STATUS", "PAYMENT_DATE","EXPIRE_DATE","PROVIDER"]

In [31]:
new_df = df.select(SELECTED_COLS)

new_df.show()

+--------------+--------------+--------------------+--------------------+--------+
|ACCOUNT_NUMBER|PAYMENT_STATUS|        PAYMENT_DATE|         EXPIRE_DATE|PROVIDER|
+--------------+--------------+--------------------+--------------------+--------+
|     038137149|       Success|2020-10-20 17:07:...|2020-10-28 18:10:...|     Ovo|
|     042103729|       Success|2015-02-28 16:44:...|2015-03-08 05:27:...|     BCA|
|     058689635|       Success|2022-04-03 13:27:...|2022-04-06 14:20:...| Permata|
|     107161965|       Success|2019-10-04 08:39:...|2019-10-08 02:01:...|     BNI|
|     014334131|        Failed|                NULL|2020-06-14 12:02:...|     BSI|
|     043583173|       Waiting|                NULL|2020-03-25 03:35:...|     Ovo|
|     021381685|       Waiting|                NULL|2017-10-12 03:15:...|     BSI|
|     119636925|       Success|2019-07-06 00:27:...|2019-07-16 05:38:...| Permata|
|     128447299|       Success|2020-10-22 21:58:...|2020-11-07 21:05:...| Mandiri|
|   

Next, we're going to filter the data when the `PROVIDER` is `BSI` and the status is `Waiting`

In [37]:
filter_new_df = new_df.filter("PROVIDER in ('BSI','Ovo') and PAYMENT_STATUS = 'Waiting'")
filter_new_df.show()

+--------------+--------------+------------+--------------------+--------+
|ACCOUNT_NUMBER|PAYMENT_STATUS|PAYMENT_DATE|         EXPIRE_DATE|PROVIDER|
+--------------+--------------+------------+--------------------+--------+
|     043583173|       Waiting|        NULL|2020-03-25 03:35:...|     Ovo|
|     021381685|       Waiting|        NULL|2017-10-12 03:15:...|     BSI|
|     067955907|       Waiting|        NULL|2018-01-23 00:35:...|     Ovo|
|     117651601|       Waiting|        NULL|2019-05-23 21:16:...|     BSI|
|     117451542|       Waiting|        NULL|2016-10-09 18:40:...|     BSI|
|     042230919|       Waiting|        NULL|2016-07-31 02:22:...|     BSI|
|     071579340|       Waiting|        NULL|2018-05-28 17:52:...|     Ovo|
|     041985553|       Waiting|        NULL|2015-08-12 03:03:...|     BSI|
|     065955077|       Waiting|        NULL|2020-05-07 08:32:...|     Ovo|
+--------------+--------------+------------+--------------------+--------+



To save it only one file, we can use this snippet code

```python
df.coalesce(numPartitions = 1).write.csv(filename)
```

---
Say we want to save the output into directory `data/output/`

In [39]:
new_df.coalesce(numPartitions = 1).write.csv("../data/output/filtered_data_single", header = True)

Then, if we want to save the output into partition files we can use without method `coalesce`

In [40]:
new_df.write.option("header", True).csv("../data/output/filtered_data_partition")

**Save to Database**
---

- To save to a database is just like when we read data from a database
- We must config the connection first and use the `jdbc` method

---
- When load to the database, PySpark can generate the table automatically
- But, you're also can build the table first and then load the data

Schema Table

```sqlrk;

CREATE TABLE public.payment_pyspark (
	"ACCOUNT_NUMBER" text NULL,
	"PAYMENT_STATUS" varchar NULL,
	"PAYMENT_DATE" datetime NULL,
	"EXPIRE_DATE" datetime NULL,
	"PROVIDER" varchar NULL,
);
```

In [43]:
# set variable for database
DB_URL = "jdbc:postgresql://pachotel_db_container:5432/pachotel"
DB_TABLE = "payment" 
DB_USER = "postgres"
DB_PASS = "cobapassword"

In [44]:
url = f"jdbc:postgresql://pachotel_db_container:5432/pachotel"
properties = {
    "user": DB_USER,
    "password": DB_PASS
}

In [46]:
new_df.write.jdbc(url = url,
                          table = "payment_pyspark",
                          mode = "overwrite",
                          properties = properties)

# <font color='blue'>Select Data using PySpark
---

- To Select Data using PySpark it's like in Pandas
- To access the data that we want, we can select the columns that we want
- Say, we want to access the flight transactional data

In [49]:
df_churns = spark \
            .read \
            .option("header", "true") \
            .csv("../data/bank_churners.csv")

df_churns.show(5)

+-----------------+------------+------+---------------+---------------+--------------+---------------+-------------+--------------+------------------------+----------------------+---------------------+------------+-------------------+---------------+--------------------+---------------+--------------+-------------------+---------------------+
|   Attrition_Flag|Customer_Age|Gender|Dependent_count|Education_Level|Marital_Status|Income_Category|Card_Category|Months_on_book|Total_Relationship_Count|Months_Inactive_12_mon|Contacts_Count_12_mon|Credit_Limit|Total_Revolving_Bal|Avg_Open_To_Buy|Total_Amt_Chng_Q4_Q1|Total_Trans_Amt|Total_Trans_Ct|Total_Ct_Chng_Q4_Q1|Avg_Utilization_Ratio|
+-----------------+------------+------+---------------+---------------+--------------+---------------+-------------+--------------+------------------------+----------------------+---------------------+------------+-------------------+---------------+--------------------+---------------+--------------+--------

- To select the columns, we can store them first in the Python list for easier use
- Then, to select the data we can use syntax `select(col_1, col_2, col_n)

In [51]:
SELECTED_COLUMNS = ["Customer_Age","Gender","Dependent_count","Education_Level","Marital_Status","Income_Category"]

selected_df_churns = df_churns.select(SELECTED_COLUMNS)
selected_df_churns.show()

+------------+------+---------------+---------------+--------------+---------------+
|Customer_Age|Gender|Dependent_count|Education_Level|Marital_Status|Income_Category|
+------------+------+---------------+---------------+--------------+---------------+
|          45|     M|              3|    High School|       Married|    $60K - $80K|
|          49|     F|              5|       Graduate|        Single| Less than $40K|
|          51|     M|              3|       Graduate|       Married|   $80K - $120K|
|          40|     F|              4|    High School|       Unknown| Less than $40K|
|          40|     M|              3|     Uneducated|       Married|    $60K - $80K|
|          44|     M|              2|       Graduate|       Married|    $40K - $60K|
|          32|     M|              0|    High School|       Unknown|    $60K - $80K|
|          37|     M|              3|     Uneducated|        Single|    $60K - $80K|
|          48|     M|              2|           NULL|        Sing

# <font color='blue'>Filter Data using PySpark
---

- To filter data in PySpark we can use the function `.filter()`
- Also, when filtering in PySpark we can access via column or SQL based
- So, when we're doing filtering data we can use **comparison operators**

<center>

| **Comparison Operators** | **Description**           |
|--------------------------|-------------------------|
| <                        | Less than             |
| >                        | More than              |
| <=                       | Less than equal to  |
| >=                       | More than equal to  |
| ==                       | Equal to             |
| !=                       | Not equal to       |

</center>

- When filtering data in PySpark we can do single or multiple filter
- When do multiple filtering we can use **boolean logic** like **and (`&`)** **or (`|`)**

**Single Filter**
---

When filtering only one column we can use this snippet code

```python
df_filtered = df.filter(df[col_1] > value)
```

In [54]:
df_filtered = df_churns.filter(df_churns['Gender'] =='F')
df_filtered.show(5)

+-----------------+------------+------+---------------+---------------+--------------+---------------+-------------+--------------+------------------------+----------------------+---------------------+------------+-------------------+---------------+--------------------+---------------+--------------+-------------------+---------------------+
|   Attrition_Flag|Customer_Age|Gender|Dependent_count|Education_Level|Marital_Status|Income_Category|Card_Category|Months_on_book|Total_Relationship_Count|Months_Inactive_12_mon|Contacts_Count_12_mon|Credit_Limit|Total_Revolving_Bal|Avg_Open_To_Buy|Total_Amt_Chng_Q4_Q1|Total_Trans_Amt|Total_Trans_Ct|Total_Ct_Chng_Q4_Q1|Avg_Utilization_Ratio|
+-----------------+------------+------+---------------+---------------+--------------+---------------+-------------+--------------+------------------------+----------------------+---------------------+------------+-------------------+---------------+--------------------+---------------+--------------+--------

**Multiple Filter**
---

When do multiple filtering in PySpark, we can use **boolean logic**

```python

# using columns based
filtered_df = df.filter((df[col_1] > value_1) & (df[col_2] == value_2))

# using sql based
filtered_df = df.filter("col_1 > value_1 AND col_2 = value_2")
```

In [55]:
df_filtered = df_churns.filter((df_churns['Gender'] =='M')& (df_churns['Education_Level'] =='Graduate'))
df_filtered.show(5)

+-----------------+------------+------+---------------+---------------+--------------+---------------+-------------+--------------+------------------------+----------------------+---------------------+------------+-------------------+---------------+--------------------+---------------+--------------+-------------------+---------------------+
|   Attrition_Flag|Customer_Age|Gender|Dependent_count|Education_Level|Marital_Status|Income_Category|Card_Category|Months_on_book|Total_Relationship_Count|Months_Inactive_12_mon|Contacts_Count_12_mon|Credit_Limit|Total_Revolving_Bal|Avg_Open_To_Buy|Total_Amt_Chng_Q4_Q1|Total_Trans_Amt|Total_Trans_Ct|Total_Ct_Chng_Q4_Q1|Avg_Utilization_Ratio|
+-----------------+------------+------+---------------+---------------+--------------+---------------+-------------+--------------+------------------------+----------------------+---------------------+------------+-------------------+---------------+--------------------+---------------+--------------+--------

# <font color='blue'>Data Transformation using PySpark
---

- In this section, we're going to learn how to do Data Wrangling or Data Transformation in PySpark
- Just like in Pandas, we can also do Data Transformation using PySpark
- There are so many tasks that we can do in Data Transformation by using PySpark
- But, in this session we will focus to:
    - Rename Column
    - Slicing Data
    - Create a New Column using the Existing Column
    - Impute Missing Values


Just like in the previous section, make sure we've already created `SparkSession`

In [3]:
# import SparkSession
from pyspark.sql import SparkSession
import pyspark

In [4]:
spark = SparkSession \
    .builder \
    .appName("Data Transformation using PySpark") \
    .config("spark.sql.legacy.timeParserPolicy", "LEGACY") \
    .getOrCreate()

In [5]:
spark

- We will be using data from the previous section
- The data is in this directory `data/materi_10/flights_filtered.csv`
- We can also create a Python function to read the data

In [7]:
def read_csv_data(filename: str) -> pyspark.sql.dataframe.DataFrame:
    """Function to read csv file using PySpark"""
    df = spark \
        .read \
        .option("header", "true") \
        .csv(filename)

    return df

In [8]:
# define data path
DATA_PATH = "../data/"

In [9]:
df_data = read_csv_data(DATA_PATH + "bank_churners.csv")

df_data.show(5) # get 5 data

+-----------------+------------+------+---------------+---------------+--------------+---------------+-------------+--------------+------------------------+----------------------+---------------------+------------+-------------------+---------------+--------------------+---------------+--------------+-------------------+---------------------+
|   Attrition_Flag|Customer_Age|Gender|Dependent_count|Education_Level|Marital_Status|Income_Category|Card_Category|Months_on_book|Total_Relationship_Count|Months_Inactive_12_mon|Contacts_Count_12_mon|Credit_Limit|Total_Revolving_Bal|Avg_Open_To_Buy|Total_Amt_Chng_Q4_Q1|Total_Trans_Amt|Total_Trans_Ct|Total_Ct_Chng_Q4_Q1|Avg_Utilization_Ratio|
+-----------------+------------+------+---------------+---------------+--------------+---------------+-------------+--------------+------------------------+----------------------+---------------------+------------+-------------------+---------------+--------------------+---------------+--------------+--------

### **Rename Column**
---

- When reading data directly from data sources, usually there's a problem
- The naming convention for column names is not proper
- For naming convention usually each person may differ, we can use camelCase or snake_case
- In this course, we will use snake_case
- To rename column names in PySpark we can use two methods:
    - `withColumnRenamed()`: can be used when rename one column only
    - `withColumnsRenamed()`: can be used when rename multiple columns

**Syntax Rename One Column**

```python
df = df.withColumnRenamed(existing = old_column, new = new_column)
```

**Syntax Rename Multiple Columns**

```python
# initiate dictionary
RENAME_COLS = {
    "old_col_1": "new_col_1",
    "old_col_2": "new_col_2",
    "old_col_n": "new_col_n"
}

df = df.withColumnsRenamed(colsMap = RENAME_COLS)
```

**Rename One Column**

By using data in variable `df_fligts`, we want to rename column `AIRLINE` to `airline`

In [ ]:
df_flights.withColumnRenamed(existing = "Dependent_count", new = "Dependent_Count").show()